 # 挂载谷歌云盘
 这一步很重要，Colab的运行原理实际上就是给你分配一台远程的带GPU的主机，所以它的原始路径不是你的谷歌云盘（也就是你的代码文件）所在的路径。所以第一步我们先要把谷歌云盘挂载带到那台远程主机上

In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# 更改运行目录

In [0]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Yolo")

# 安装

In [0]:
! git clone https://github.com/YunYang1994/tensorflow-yolov3.git

Cloning into 'tensorflow-yolov3'...
remote: Enumerating objects: 1750, done.
remote: Total 1750 (delta 0), reused 0 (delta 0), pack-reused 1750
Receiving objects: 100% (1750/1750), 62.86 MiB | 18.20 MiB/s, done.
Resolving deltas: 100% (1058/1058), done.


不能用pip安装tensorflow，官方提示了会与GPU冲突，尽量使用自带的。 [Avoid Using pip install with GPUs and TPUs](https://colab.research.google.com/notebooks/tensorflow_version.ipynb#scrollTo=8UvRkm1JGUrk)
- numpy==1.15.1
- Pillow==5.3.0
- scipy==1.1.0
- ~~tensorflow-gpu==1.11.0~~
- wget==3.2
- seaborn==0.9.0

In [0]:
! pip install numpy==1.15.1
! pip install Pillow==5.3.0
! pip install scipy==1.1.0
! pip install wget==3.2
! pip install seaborn==0.9.0

In [0]:
% cd tensorflow-yolov3/checkpoint
! wget https://github.com/YunYang1994/tensorflow-yolov3/releases/download/v1.0/yolov3_coco.tar.gz
! tar -xvf yolov3_coco.tar.gz

# 运行

In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
%cd tensorflow-yolov3
%ls

/content/gdrive/My Drive/Colab Notebooks/Yolo/tensorflow-yolov3
checkpoint/        from_darknet_weights_to_ckpt.py  scripts/
convert_weight.py  from_darknet_weights_to_pb.py    test_material/
core/              image_demo.py                    train.py
data/              LICENSE                          video_demo.py
docs/              LICENSE.fuck                     yolov3_coco.pb
evaluate.py        mAP/
freeze_graph.py    README.md


In [0]:
! python ./convert_weight.py

In [0]:
! python ./freeze_graph.py

# 测试

In [16]:
# from google.colab import drive
# drive.mount('/content/gdrive')
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Yolo")
# %tensorflow_version 1.x
# %cd tensorflow-yolov3
# %ls

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/Yolo/tensorflow-yolov3
1.png              from_darknet_weights_to_ckpt.py  mAP/
checkpoint/        from_darknet_weights_to_pb.py    README.md
convert_weight.py  image_demo.py                    scripts/
core/              image_test_result/               test_material/
data/              input.mp4                        train.py
docs/              input.png                        video_demo.py
evaluate.py        LICENSE                          yolov3_coco.pb
freeze_graph.py    LICENSE.fuck


## 输入视频，输出视频+每帧图像

In [0]:
import cv2
import time
import numpy as np
import core.utils as utils
import tensorflow as tf
from PIL import Image


return_elements = ["input/input_data:0", "pred_sbbox/concat_2:0", "pred_mbbox/concat_2:0", "pred_lbbox/concat_2:0"]
# 模型pb文件路径
pb_file = "./yolov3_coco.pb"
# 视频图像路径
video_path= "./input.mp4"
# 摄像头输入端
# video_path = 0
#保存视频路径
save_path="./result.avi"
#是否保存检测结果视频
storable=True
# 目标检测类别总数
num_classes = 80
# 输入图像的尺寸
input_size = 416
#从video_path中加载视频
#若video_path=0加载照相机中视频若video_path="str"加载str路径下的视频
vid = cv2.VideoCapture(video_path)
#获得fps值
fps = vid.get(cv2.CAP_PROP_FPS)
#获取vid的每一帧图像大小
size = (int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)), int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT)))
#声明保存视频的路径、视频编码格式、fps、图像尺寸大小
videoWriter = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc('I', '4', '2', '0'), fps, size)
print(111)

#tf.graph(),定义计算图
# 计算图用于构建网络，本身不进行任何实际的计算
graph = tf.Graph()
# 从pb文件将计算图导入到当前默认图中
return_tensors = utils.read_pb_return_tensors(graph, pb_file, return_elements)
with tf.Session(graph=graph) as sess:
  while True:
    #按帧读取视频,vid.read()返回两个值,
    #return是bool值,如果读取帧正确则返回True，如果文件读取到结尾,他的返回值就为False
    #fram是三维矩阵，就是每一帧的图像
    return_value, frame = vid.read()
    if return_value:
      #cv2.VideoCapture()读取后的图像为BGR格式
      #将每一帧BGR图像转换成RGB图像,便于图像处理
      frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      #array转换成image
      image = Image.fromarray(frame)
    else:
      # raise ValueError("No image!")
      break
    #获取图像尺寸大小
    frame_size = frame.shape[:2]
    #图像预处理,这块图像其实没有发生变化
    #因为target_size和frame.shape[:2]是一样的
    image_data = utils.image_preporcess(np.copy(frame), [input_size, input_size])
    #增加一维，就是batch_size维,默认该维度为1
    image_data = image_data[np.newaxis, ...]
    #获取每一帧处理前的时间戳
    prev_time = time.time()
    #得到三种bounding box
    pred_sbbox, pred_mbbox, pred_lbbox = sess.run(
    [return_tensors[1], return_tensors[2], return_tensors[3]],
    feed_dict={ return_tensors[0]: image_data})
    #将预测结果组成一个矩阵
    pred_bbox = np.concatenate([np.reshape(pred_sbbox, (-1, 5 + num_classes)),
    np.reshape(pred_mbbox, (-1, 5 + num_classes)),
    np.reshape(pred_lbbox, (-1, 5 + num_classes))], axis=0)
    #TODO:
    bboxes = utils.postprocess_boxes(pred_bbox, frame_size, input_size, 0.3)
    #非极大值抑制，IOU的阈值设为0.45
    bboxes = utils.nms(bboxes, 0.45, method='nms')
    #TODO:
    #得到的结果是一张张图片
    image = utils.draw_bbox(frame, bboxes)
    #获得每一帧处理后的时间戳
    curr_time = time.time()
    #计算每一帧处理时间
    exec_time = curr_time - prev_time
    result = np.asarray(image)
    # 输出每一帧处理时间
    print("time: %.2f ms" %(1000*exec_time))
    #图片的标题
    # cv2.namedWindow("result", cv2.WINDOW_AUTOSIZE)
    #将RGB格式转化为BGR格式，便于cv2显示
    # result = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    #显示图像
    cv2.imwrite(str("./test_result/"+str(curr_time)+".jpg"), result) 
    # cv2.imshow("result", result)
    #保存图像
    if storable:
      videoWriter.write(result)
    #键盘延迟1ms按'q'键退出
    if cv2.waitKey(1) & 0xFF == ord('q'):break
print("---------------------------------结束---------------------------------------")

## 输入**图片**， 输出图片

In [2]:
import cv2
import numpy as np
import core.utils as utils
import tensorflow as tf
from PIL import Image
import time
import math

def fire_hydrants_classification(self, original_image):

    return_elements = ["input/input_data:0", "pred_sbbox/concat_2:0", "pred_mbbox/concat_2:0", "pred_lbbox/concat_2:0"]
    pb_file         = "./yolov3_coco.pb"
    image_path      = "./input.png"
    num_classes     = 80
    input_size      = 416
    graph           = tf.Graph()

    # original_image = cv2.imread(image_path)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    original_image_size = original_image.shape[:2]
    image_data = utils.image_preporcess(np.copy(original_image), [input_size, input_size])
    image_data = image_data[np.newaxis, ...]

    return_tensors = utils.read_pb_return_tensors(graph, pb_file, return_elements)


    with tf.Session(graph=graph) as sess:
        pred_sbbox, pred_mbbox, pred_lbbox = sess.run(
            [return_tensors[1], return_tensors[2], return_tensors[3]],
                    feed_dict={ return_tensors[0]: image_data})

    pred_bbox = np.concatenate([np.reshape(pred_sbbox, (-1, 5 + num_classes)),
                                np.reshape(pred_mbbox, (-1, 5 + num_classes)),
                                np.reshape(pred_lbbox, (-1, 5 + num_classes))], axis=0)

    bboxes = utils.postprocess_boxes(pred_bbox, original_image_size, input_size, 0.3)
    bboxes = utils.nms(bboxes, 0.45, method='nms')


    # 裁剪图像
    for i in bboxes:
      print(i)
      if i[5] == 10.0:
        # 向上取整不需要判断是否越界，最大值只会等于shape
        # 裁剪坐标为[y0:y1, x0:x1]
        cropped_image = original_image[int(i[1]):math.ceil(i[3]),int(i[0]):math.ceil(i[2])]
        break


    # 转换RGB格式
    cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
    return cropped_image
    # image = Image.fromarray(image)
    # 保存cropped_image图像
    # cv2.imwrite("./image_test_result/cropped_image.png", cropped_image)

    # 画框框 draw_bbox
    image = utils.draw_bbox(original_image, bboxes)
    # 转换RGB格式
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # 保存image图像
    # cv2.imwrite("./image_test_result/full_image.png", image)

if __name__ == '__main__':
    image_path = "./input.png"
    original_image = cv2.imread(image_path)
    cropped_image = fire_hydrants_classification(original_image)
    cv2.imwrite("./image_test_result/cropped_image.png", cropped_image)

TypeError: fire_hydrants_classification() missing 1 required positional argument: 'original_image'

## 输入视频，cropped图像

In [5]:
import cv2
import time
import numpy as np
import core.utils as utils
import tensorflow as tf
from PIL import Image
import math

def fire_hydrants_classification(original_image):

    return_elements = ["input/input_data:0", "pred_sbbox/concat_2:0", "pred_mbbox/concat_2:0", "pred_lbbox/concat_2:0"]
    pb_file         = "./yolov3_coco.pb"
    image_path      = "./input.png"
    num_classes     = 80
    input_size      = 416
    graph           = tf.Graph()

    # original_image = cv2.imread(image_path)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    original_image_size = original_image.shape[:2]
    image_data = utils.image_preporcess(np.copy(original_image), [input_size, input_size])
    image_data = image_data[np.newaxis, ...]

    return_tensors = utils.read_pb_return_tensors(graph, pb_file, return_elements)


    with tf.Session(graph=graph) as sess:
        pred_sbbox, pred_mbbox, pred_lbbox = sess.run(
            [return_tensors[1], return_tensors[2], return_tensors[3]],
                    feed_dict={ return_tensors[0]: image_data})

    pred_bbox = np.concatenate([np.reshape(pred_sbbox, (-1, 5 + num_classes)),
                                np.reshape(pred_mbbox, (-1, 5 + num_classes)),
                                np.reshape(pred_lbbox, (-1, 5 + num_classes))], axis=0)

    bboxes = utils.postprocess_boxes(pred_bbox, original_image_size, input_size, 0.3)
    bboxes = utils.nms(bboxes, 0.45, method='nms')


    # 裁剪图像
    for i in bboxes:
      print(i)
      if i[5] == 10.0:
        # 向上取整不需要判断是否越界，最大值只会等于shape
        # 裁剪坐标为[y0:y1, x0:x1]
        cropped_image = original_image[int(i[1]):math.ceil(i[3]),int(i[0]):math.ceil(i[2])]
        break


    # 转换RGB格式
    cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
    return cropped_image
    # image = Image.fromarray(image)
    # 保存cropped_image图像
    # cv2.imwrite("./image_test_result/cropped_image.png", cropped_image)

    # 画框框 draw_bbox
    image = utils.draw_bbox(original_image, bboxes)
    # 转换RGB格式
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # 保存image图像
    # cv2.imwrite("./image_test_result/full_image.png", image)



return_elements = ["input/input_data:0", "pred_sbbox/concat_2:0", "pred_mbbox/concat_2:0", "pred_lbbox/concat_2:0"]
# 模型pb文件路径
pb_file = "./yolov3_coco.pb"
# 视频图像路径
video_path= "./input.mp4"
# 摄像头输入端
# video_path = 0
#保存视频路径
save_path="./result.avi"
#是否保存检测结果视频
storable=True
# 目标检测类别总数
num_classes = 80
# 输入图像的尺寸
input_size = 416
#从video_path中加载视频
#若video_path=0加载照相机中视频若video_path="str"加载str路径下的视频
vid = cv2.VideoCapture(video_path)
#获得fps值
fps = vid.get(cv2.CAP_PROP_FPS)
#获取vid的每一帧图像大小
size = (int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)), int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT)))
#声明保存视频的路径、视频编码格式、fps、图像尺寸大小
videoWriter = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc('I', '4', '2', '0'), fps, size)
print(time.asctime( time.localtime(time.time())))

count = 0

#tf.graph(),定义计算图
# 计算图用于构建网络，本身不进行任何实际的计算
graph = tf.Graph()
# 从pb文件将计算图导入到当前默认图中
return_tensors = utils.read_pb_return_tensors(graph, pb_file, return_elements)
with tf.Session(graph=graph) as sess:
  while True:
    #按帧读取视频,vid.read()返回两个值,
    #return是bool值,如果读取帧正确则返回True，如果文件读取到结尾,他的返回值就为False
    #fram是三维矩阵，就是每一帧的图像
    return_value, frame = vid.read()
    if return_value:
      #cv2.VideoCapture()读取后的图像为BGR格式
      #将每一帧BGR图像转换成RGB图像,便于图像处理
      frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      #array转换成image
      # image = Image.fromarray(frame)
    else:
      # raise ValueError("No image!")
      break
    #获取每一帧处理前的时间戳
    prev_time = time.time()

    image = fire_hydrants_classification(frame)

    #获得每一帧处理后的时间戳
    curr_time = time.time()
    #计算每一帧处理时间
    exec_time = curr_time - prev_time
    result = np.asarray(image)
    # 输出每一帧处理时间
    print("time: %.2f ms" %(1000*exec_time))
    #图片的标题
    # cv2.namedWindow("result", cv2.WINDOW_AUTOSIZE)
    #将RGB格式转化为BGR格式，便于cv2显示
    # result = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    #显示图像
    count += 1
    cv2.imwrite(str("./image_test_result/"+str(count)+".jpg"), result)
    # cv2.imshow("result", result)
    #保存图像
    # if storable:
    #   videoWriter.write(result)
    #键盘延迟1ms按'q'键退出
    if cv2.waitKey(1) & 0xFF == ord('q'):break
print("---------------------------------结束---------------------------------------")




Mon Jun  8 02:34:45 2020
[ 1.59567916  0.         42.44668579 32.86462784  0.84042156  2.        ]


UnboundLocalError: local variable 'cropped_image' referenced before assignment